In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
import sys
import os
sys.path.append(os.path.abspath('../'))
sys.path.append(os.path.abspath(''))

In [2]:



import pandas as pd
import numpy as np
import jsonlines
import seaborn as sns
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import torch.nn as nn
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils


from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from importlib import reload
pd.set_option('display.max_rows', 500)
pd.set_option('display.float_format', '{:0.3f}'.format)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.options.display.width = 0
import warnings
warnings.filterwarnings('ignore')
import facebook_hateful_memes_detector
reload(facebook_hateful_memes_detector)

from facebook_hateful_memes_detector.utils import read_json_lines_into_df, in_notebook
from facebook_hateful_memes_detector.models import LangFeaturesModel
from facebook_hateful_memes_detector.preprocessing import TextImageDataset, my_collate, get_datasets, get_basic_image_transforms
from facebook_hateful_memes_detector.training import *



Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
preprocess = get_basic_image_transforms()

In [5]:
data = get_datasets(data_dir="../data/", train_text_transform=None, train_image_transform=preprocess, 
                 test_text_transform=None, test_image_transform=preprocess, 
                 cache_images = True, use_images = False, dev=False)

https://discuss.pytorch.org/t/how-to-implement-torch-optim-lr-scheduler-cosineannealinglr/28797/11

## LangFeatures GRU

In [8]:
model_fn = model_builder(LangFeaturesModel, 
                         dict(classifer_dims=1024, num_classes=2, 
                              gaussian_noise=0.0, dropout=0.0, use_as_submodel=False,
                              gru_layers=1, gru_dropout=0.0, gru_dims=256, capabilities=["snlp"]),
                         optimiser_class=torch.optim.Adam, optimiser_params=dict(lr=0.001, weight_decay=1e-9))



n_tests = 1
batch_size=512
epochs = 5
results, prfs = train_validate_ntimes(model_fn, data, n_tests, batch_size, epochs)


2020-05-30 11:00:26 INFO: Loading these models for language: en (English):
| Processor | Package   |
-------------------------
| tokenize  | ewt       |
| pos       | ewt       |
| lemma     | ewt       |
| depparse  | ewt       |
| ner       | ontonotes |

2020-05-30 11:00:26 INFO: Use device: cpu
2020-05-30 11:00:26 INFO: Loading: tokenize
2020-05-30 11:00:26 INFO: Loading: pos
2020-05-30 11:00:27 INFO: Loading: lemma
2020-05-30 11:00:27 INFO: Loading: depparse
2020-05-30 11:00:28 INFO: Loading: ner
2020-05-30 11:00:29 INFO: Done loading processors!


In [7]:
results
prfs

train   val
mean_or_std metric               
mean        f1_micro  0.615 0.611
            map       0.479 0.476
            accuracy  0.615 0.611
            auc       0.622 0.622
std         f1_micro  0.000 0.000
            map       0.000 0.000
            accuracy  0.000 0.000
            auc       0.000 0.000

train                             val                      
    precision recall    f1  supoort precision recall    f1  supoort
neg     0.712  0.672 0.692 4372.000     0.706  0.663 0.684 1078.000
pos     0.464  0.512 0.487 2428.000     0.472  0.521 0.495  622.000

In [ ]:
submission = train_and_predict(model_fn, data, batch_size, epochs)
submission.to_csv("submission.csv",index=False)
submission.head(3)
